In [ ]:
%matplotlib inline

import os
import logging

from configurations import importer
import django
from django.db import connection
import dotenv
import pandas as pd

dotenv.read_dotenv()
os.environ['DJANGO_SETTINGS_MODULE'] = 'calltospeakers.settings'
os.environ['DJANGO_CONFIGURATION'] = 'Development'
importer.install()
django.setup()

In [ ]:
def dictfetchall(cursor):
    "Returns all rows from a cursor as a dict"
    desc = cursor.description
    return [
        dict(zip([col[0] for col in desc], row))
        for row in cursor.fetchall()
    ]

In [ ]:
cursor = connection.cursor()
cursor.execute("""
CREATE OR REPLACE VIEW wow_talk AS
SELECT date_trunc('week', created) AT TIME ZONE 'PST' as week, 
       count(*) as talks
FROM cfp_talk
GROUP BY date_trunc('week', created)
ORDER BY date_trunc('week', created) ASC
""")

cursor.execute("""
SELECT week,
  cast(talks as numeric),
  lag(talks, 1) over w as previous,
  round((cast(talks as numeric) / (lag(talks, 1) over w) - 1) * 100, 1) as growth
FROM wow_talk
WINDOW w AS (ORDER BY week ASC)
ORDER BY week ASC
""")
talks_wow = pd.DataFrame.from_records(dictfetchall(cursor), index='week')
talks_wow

In [ ]:
talks_wow[1:-1]['growth'].astype(float).plot()

In [ ]:
cursor = connection.cursor()
cursor.execute("""
CREATE OR REPLACE VIEW wow_user AS
SELECT date_trunc('week', date_joined) AT TIME ZONE 'PST' as week, 
       count(*) as users
FROM auth_user
GROUP BY date_trunc('week', date_joined)
ORDER BY date_trunc('week', date_joined) ASC
""")

cursor.execute("""
SELECT week,
  cast(users as numeric),
  lag(users, 1) over w as previous,
  round((cast(users as numeric) / (lag(users, 1) over w) - 1) * 100, 1) as growth
FROM wow_user
WINDOW w AS (ORDER BY week ASC)
ORDER BY week ASC
""")
users_wow = pd.DataFrame.from_records(dictfetchall(cursor), index='week')
users_wow

In [ ]:
users_wow[1:-1]['growth'].astype(float).plot()